In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())

        rgb = np.dstack((r, g, b))
        images.append(rgb)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [6]:
y = train_df.is_iceberg.values
print(y[:5])
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
import pickle
from sklearn.metrics import log_loss

def lr_f(epoch):
    if epoch < 15:
        return 0.0005
    elif epoch < 30:
        return 0.0001
    elif epoch < 45:
        return 0.00005
    else:
        return 0.00001

[0 0 1 0 0]


In [7]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):

    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 3
    
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def create_incept_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # bn
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = ZeroPadding2D((1, 1))(x)
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    x = ZeroPadding2D((1, 1))(x)
    x = conv2d_bn(x, 64, 1, 1, padding='valid')
    x = conv2d_bn(x, 64, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

     # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')
    x = ZeroPadding2D((1, 1))(x)

    # mixed 1: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')
    x = ZeroPadding2D((1, 1))(x)

    # mixed 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')
    x = ZeroPadding2D((1, 1))(x)
    
    # mixed 3: 17 x 17 x 768
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(
        branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = layers.concatenate(
        [branch3x3, branch3x3dbl, branch_pool], axis=channel_axis, name='mixed3')

    x = Conv2D(96, 3, strides=2, padding='same',activation='relu')(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

model model


In [8]:
def kfold_train(fold_cnt=3,rnd=99):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_incept_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.69151, saving model to best_m.h5
 - 10s - loss: 0.7064 - acc: 0.6154 - val_loss: 0.6915 - val_acc: 0.5389
Epoch 2/120
Epoch 00002: val_loss did not improve
 - 6s - loss: 0.4888 - acc: 0.7718 - val_loss: 0.7333 - val_acc: 0.5389
Epoch 3/120
Epoch 00003: val_loss did not improve
 - 6s - loss: 0.4336 - acc: 0.7941 - val_loss: 0.7453 - val_acc: 0.5389
Epoch 4/120
Epoch 00004: val_loss improved from 0.69151 to 0.68049, saving model to best_m.h5
 - 6s - loss: 0.4203 - acc: 0.8044 - val_loss: 0.6805 - val_acc: 0.5421
Epoch 5/120
Epoch 00005: val_loss improved from 0.68049 to 0.58927, saving model to best_m.h5
 - 6s - loss: 0.4304 - acc: 0.7958 - val_loss: 0.5893 - val_acc: 0.5857
Epoch 6/120
Epoch 00006: val_loss improved from 0.58927 to 0.48550, saving model to best_m.h5
 - 6s - loss: 0.3917 - acc: 0.8292 - val_loss: 0.4855 - val_acc: 0.7726
Epoch 7/120
Epoch 00007: val_loss improved from 0.48550 to 0.35620, saving model to best_m.h5
 

Epoch 14/120
Epoch 00014: val_loss did not improve
 - 6s - loss: 0.3558 - acc: 0.8359 - val_loss: 0.4589 - val_acc: 0.7913
Epoch 15/120
Epoch 00015: val_loss did not improve
 - 6s - loss: 0.3385 - acc: 0.8344 - val_loss: 0.2753 - val_acc: 0.8692
Epoch 16/120
Epoch 00016: val_loss improved from 0.26983 to 0.24955, saving model to best_m.h5
 - 6s - loss: 0.3132 - acc: 0.8537 - val_loss: 0.2495 - val_acc: 0.9003
Epoch 17/120
Epoch 00017: val_loss improved from 0.24955 to 0.23175, saving model to best_m.h5
 - 6s - loss: 0.2794 - acc: 0.8726 - val_loss: 0.2317 - val_acc: 0.9159
Epoch 18/120
Epoch 00018: val_loss improved from 0.23175 to 0.22925, saving model to best_m.h5
 - 6s - loss: 0.2985 - acc: 0.8552 - val_loss: 0.2293 - val_acc: 0.8847
Epoch 19/120
Epoch 00019: val_loss improved from 0.22925 to 0.22861, saving model to best_m.h5
 - 6s - loss: 0.3079 - acc: 0.8566 - val_loss: 0.2286 - val_acc: 0.9128
Epoch 20/120
Epoch 00020: val_loss improved from 0.22861 to 0.21992, saving model to b

Epoch 00001: val_loss improved from inf to 0.71297, saving model to best_m.h5
 - 19s - loss: 0.6743 - acc: 0.6734 - val_loss: 0.7130 - val_acc: 0.5327
Epoch 2/120
Epoch 00002: val_loss did not improve
 - 6s - loss: 0.4636 - acc: 0.7784 - val_loss: 0.7492 - val_acc: 0.5327
Epoch 3/120
Epoch 00003: val_loss did not improve
 - 6s - loss: 0.4200 - acc: 0.8086 - val_loss: 1.0043 - val_acc: 0.5327
Epoch 4/120
Epoch 00004: val_loss did not improve
 - 6s - loss: 0.4099 - acc: 0.8029 - val_loss: 1.3124 - val_acc: 0.5327
Epoch 5/120
Epoch 00005: val_loss did not improve
 - 6s - loss: 0.3652 - acc: 0.8271 - val_loss: 1.2926 - val_acc: 0.5327
Epoch 6/120
Epoch 00006: val_loss improved from 0.71297 to 0.68425, saving model to best_m.h5
 - 6s - loss: 0.3634 - acc: 0.8365 - val_loss: 0.6842 - val_acc: 0.5421
Epoch 7/120
Epoch 00007: val_loss improved from 0.68425 to 0.39181, saving model to best_m.h5
 - 6s - loss: 0.3472 - acc: 0.8320 - val_loss: 0.3918 - val_acc: 0.8006
Epoch 8/120
Epoch 00008: val_

Epoch 63/120
Epoch 00063: val_loss did not improve
 - 6s - loss: 0.1915 - acc: 0.9219 - val_loss: 0.2401 - val_acc: 0.8910
Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.69875, saving model to best_m.h5
 - 26s - loss: 0.7067 - acc: 0.6427 - val_loss: 0.6988 - val_acc: 0.4860
Epoch 2/120
Epoch 00002: val_loss did not improve
 - 6s - loss: 0.5204 - acc: 0.7456 - val_loss: 0.8960 - val_acc: 0.4860
Epoch 3/120
Epoch 00003: val_loss did not improve
 - 6s - loss: 0.4703 - acc: 0.7597 - val_loss: 1.3599 - val_acc: 0.4860
Epoch 4/120
Epoch 00004: val_loss did not improve
 - 6s - loss: 0.4242 - acc: 0.7968 - val_loss: 2.0140 - val_acc: 0.4860
Epoch 5/120
Epoch 00005: val_loss did not improve
 - 6s - loss: 0.4329 - acc: 0.8084 - val_loss: 1.0839 - val_acc: 0.4860
Epoch 6/120
Epoch 00006: val_loss improved from 0.69875 to 0.60864, saving model to best_m.h5
 - 6s - loss: 0.4172 - acc: 0.8044 - val_loss: 0.6086 - val_acc: 0.5576
Epoch 7/120
Epoch 00007: val_loss improved from 0.60864 to 0

Epoch 2/120
Epoch 00002: val_loss did not improve
 - 6s - loss: 0.4915 - acc: 0.7563 - val_loss: 0.9883 - val_acc: 0.5312
Epoch 3/120
Epoch 00003: val_loss did not improve
 - 6s - loss: 0.4516 - acc: 0.7765 - val_loss: 1.2898 - val_acc: 0.5312
Epoch 4/120
Epoch 00004: val_loss did not improve
 - 6s - loss: 0.4376 - acc: 0.8055 - val_loss: 0.8213 - val_acc: 0.5312
Epoch 5/120
Epoch 00005: val_loss improved from 0.69040 to 0.58113, saving model to best_m.h5
 - 6s - loss: 0.4255 - acc: 0.7977 - val_loss: 0.5811 - val_acc: 0.5656
Epoch 6/120
Epoch 00006: val_loss improved from 0.58113 to 0.43974, saving model to best_m.h5
 - 6s - loss: 0.3829 - acc: 0.8289 - val_loss: 0.4397 - val_acc: 0.7969
Epoch 7/120
Epoch 00007: val_loss improved from 0.43974 to 0.34853, saving model to best_m.h5
 - 6s - loss: 0.3812 - acc: 0.8226 - val_loss: 0.3485 - val_acc: 0.8500
Epoch 8/120
Epoch 00008: val_loss did not improve
 - 6s - loss: 0.3296 - acc: 0.8375 - val_loss: 0.7979 - val_acc: 0.6562
Epoch 9/120
Ep

Epoch 62/120
Epoch 00062: val_loss did not improve
 - 6s - loss: 0.1818 - acc: 0.9266 - val_loss: 0.2280 - val_acc: 0.9125
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 6s - loss: 0.1910 - acc: 0.9203 - val_loss: 0.2272 - val_acc: 0.9156
Epoch 64/120
Epoch 00064: val_loss improved from 0.22612 to 0.22608, saving model to best_m.h5
 - 6s - loss: 0.2045 - acc: 0.9195 - val_loss: 0.2261 - val_acc: 0.9156
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 6s - loss: 0.2103 - acc: 0.9016 - val_loss: 0.2263 - val_acc: 0.9094
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 6s - loss: 0.2022 - acc: 0.9196 - val_loss: 0.2262 - val_acc: 0.9187
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 6s - loss: 0.1849 - acc: 0.9226 - val_loss: 0.2297 - val_acc: 0.9219
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 6s - loss: 0.1898 - acc: 0.9172 - val_loss: 0.2292 - val_acc: 0.9187
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 6s - loss: 0.1808 - acc: 0.9164 - val_los

In [9]:
with open('../features/incept_aug4_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/incept_aug4_sub.csv', index=False)
# deep2 0.2110
# deep3 0.2119

# pre 2290
# new 2240

0.224029977288
         id  is_iceberg
0  5941774d    0.381580
1  4023181e    0.371532
2  b20200e4    0.250473
3  e7f018bb    0.998929
4  4371c8c3    0.201019


In [10]:
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    
    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    
    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    bn_axis = 3
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = ZeroPadding2D((1, 1))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    x = ZeroPadding2D((1, 1))(x)
    
    x = Conv2D(128, 3, strides=2, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [13]:
def kfold_train(fold_cnt=3,rnd=999):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_resnet_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 30, verbose = 0, mode= 'min')
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.69470, saving model to best_m.h5
 - 60s - loss: 0.9460 - acc: 0.5997 - val_loss: 0.6947 - val_acc: 0.4735
Epoch 2/120
Epoch 00002: val_loss improved from 0.69470 to 0.69363, saving model to best_m.h5
 - 7s - loss: 0.6006 - acc: 0.6505 - val_loss: 0.6936 - val_acc: 0.5265
Epoch 3/120
Epoch 00003: val_loss improved from 0.69363 to 0.68148, saving model to best_m.h5
 - 7s - loss: 0.5950 - acc: 0.6755 - val_loss: 0.6815 - val_acc: 0.5265
Epoch 4/120
Epoch 00004: val_loss improved from 0.68148 to 0.66741, saving model to best_m.h5
 - 7s - loss: 0.5895 - acc: 0.6708 - val_loss: 0.6674 - val_acc: 0.5265
Epoch 5/120
Epoch 00005: val_loss improved from 0.66741 to 0.60875, saving model to best_m.h5
 - 7s - loss: 0.5317 - acc: 0.7044 - val_loss: 0.6087 - val_acc: 0.5265
Epoch 6/120
Epoch 00006: val_loss did not improve
 - 7s - loss: 0.5495 - acc: 0.7181 - val_loss: 0.6829 - val_acc: 0.5514
Epoch 7/120
Epoch 00007: val_loss improved from 0.6

Epoch 63/120
Epoch 00063: val_loss did not improve
 - 7s - loss: 0.2698 - acc: 0.8586 - val_loss: 0.2471 - val_acc: 0.9034
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 7s - loss: 0.2729 - acc: 0.8781 - val_loss: 0.2444 - val_acc: 0.9034
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 7s - loss: 0.3244 - acc: 0.8585 - val_loss: 0.2586 - val_acc: 0.8910
Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.68965, saving model to best_m.h5
 - 62s - loss: 1.0577 - acc: 0.6154 - val_loss: 0.6897 - val_acc: 0.5327
Epoch 2/120
Epoch 00002: val_loss did not improve
 - 7s - loss: 0.5996 - acc: 0.6757 - val_loss: 0.6952 - val_acc: 0.5327
Epoch 3/120
Epoch 00003: val_loss did not improve
 - 7s - loss: 0.5619 - acc: 0.7034 - val_loss: 0.6943 - val_acc: 0.5327
Epoch 4/120
Epoch 00004: val_loss improved from 0.68965 to 0.65973, saving model to best_m.h5
 - 7s - loss: 0.5470 - acc: 0.7333 - val_loss: 0.6597 - val_acc: 0.5327
Epoch 5/120
Epoch 00005: val_loss improved from 0.65973 to

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.69182, saving model to best_m.h5
 - 64s - loss: 1.1470 - acc: 0.5790 - val_loss: 0.6918 - val_acc: 0.5202
Epoch 2/120
Epoch 00002: val_loss did not improve
 - 7s - loss: 0.6351 - acc: 0.6507 - val_loss: 0.6919 - val_acc: 0.5202
Epoch 3/120
Epoch 00003: val_loss improved from 0.69182 to 0.69082, saving model to best_m.h5
 - 8s - loss: 0.6028 - acc: 0.6786 - val_loss: 0.6908 - val_acc: 0.5202
Epoch 4/120
Epoch 00004: val_loss improved from 0.69082 to 0.66788, saving model to best_m.h5
 - 8s - loss: 0.5685 - acc: 0.7058 - val_loss: 0.6679 - val_acc: 0.5202
Epoch 5/120
Epoch 00005: val_loss improved from 0.66788 to 0.62576, saving model to best_m.h5
 - 8s - loss: 0.5605 - acc: 0.7226 - val_loss: 0.6258 - val_acc: 0.5202
Epoch 6/120
Epoch 00006: val_loss improved from 0.62576 to 0.51077, saving model to best_m.h5
 - 8s - loss: 0.5101 - acc: 0.7320 - val_loss: 0.5108 - val_acc: 0.6698
Epoch 7/120
Epoch 00007: val_loss improved from 0.5

Epoch 62/120
Epoch 00062: val_loss did not improve
 - 7s - loss: 0.2601 - acc: 0.8859 - val_loss: 0.2911 - val_acc: 0.8442
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 7s - loss: 0.2491 - acc: 0.8930 - val_loss: 0.2910 - val_acc: 0.8442
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 7s - loss: 0.2356 - acc: 0.8865 - val_loss: 0.2916 - val_acc: 0.8411
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 7s - loss: 0.2327 - acc: 0.8896 - val_loss: 0.2983 - val_acc: 0.8442
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 7s - loss: 0.2457 - acc: 0.8873 - val_loss: 0.2906 - val_acc: 0.8474
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 7s - loss: 0.2401 - acc: 0.8937 - val_loss: 0.2923 - val_acc: 0.8380
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 7s - loss: 0.2369 - acc: 0.8828 - val_loss: 0.2978 - val_acc: 0.8380
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 7s - loss: 0.2104 - acc: 0.9131 - val_loss: 0.3049 - val_acc: 0.8411
Epoch 70/120
Epo

Epoch 50/120
Epoch 00050: val_loss did not improve
 - 7s - loss: 0.2557 - acc: 0.8781 - val_loss: 0.2689 - val_acc: 0.8785
Epoch 51/120
Epoch 00051: val_loss improved from 0.26567 to 0.26457, saving model to best_m.h5
 - 7s - loss: 0.3043 - acc: 0.8441 - val_loss: 0.2646 - val_acc: 0.8785
Epoch 52/120
Epoch 00052: val_loss did not improve
 - 7s - loss: 0.2724 - acc: 0.8740 - val_loss: 0.2727 - val_acc: 0.8816
Epoch 53/120
Epoch 00053: val_loss improved from 0.26457 to 0.26283, saving model to best_m.h5
 - 8s - loss: 0.2750 - acc: 0.8672 - val_loss: 0.2628 - val_acc: 0.8723
Epoch 54/120
Epoch 00054: val_loss did not improve
 - 7s - loss: 0.2716 - acc: 0.8773 - val_loss: 0.2674 - val_acc: 0.8754
Epoch 55/120
Epoch 00055: val_loss did not improve
 - 7s - loss: 0.2799 - acc: 0.8638 - val_loss: 0.2807 - val_acc: 0.8785
Epoch 56/120
Epoch 00056: val_loss did not improve
 - 7s - loss: 0.2835 - acc: 0.8828 - val_loss: 0.2785 - val_acc: 0.8754
Epoch 57/120
Epoch 00057: val_loss did not improve


Epoch 114/120
Epoch 00114: val_loss did not improve
 - 7s - loss: 0.2547 - acc: 0.8750 - val_loss: 0.2782 - val_acc: 0.8785
Epoch 115/120
Epoch 00115: val_loss did not improve
 - 7s - loss: 0.2346 - acc: 0.8894 - val_loss: 0.3027 - val_acc: 0.8629
Epoch 116/120
Epoch 00116: val_loss did not improve
 - 7s - loss: 0.2314 - acc: 0.8898 - val_loss: 0.2663 - val_acc: 0.8505
Epoch 117/120
Epoch 00117: val_loss did not improve
 - 7s - loss: 0.2197 - acc: 0.8961 - val_loss: 0.2695 - val_acc: 0.8692
Epoch 118/120
Epoch 00118: val_loss did not improve
 - 7s - loss: 0.2316 - acc: 0.8937 - val_loss: 0.2579 - val_acc: 0.8567
Epoch 119/120
Epoch 00119: val_loss did not improve
 - 7s - loss: 0.2390 - acc: 0.8961 - val_loss: 0.2590 - val_acc: 0.8629
Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.69218, saving model to best_m.h5
 - 73s - loss: 0.9433 - acc: 0.6047 - val_loss: 0.6922 - val_acc: 0.4844
Epoch 2/120
Epoch 00002: val_loss did not improve
 - 7s - loss: 0.6204 - acc: 0.6492 - val_lo

 - 8s - loss: 0.2777 - acc: 0.8711 - val_loss: 0.2487 - val_acc: 0.8812
Epoch 57/120
Epoch 00057: val_loss did not improve
 - 7s - loss: 0.2893 - acc: 0.8609 - val_loss: 0.2616 - val_acc: 0.8938
Epoch 58/120
Epoch 00058: val_loss did not improve
 - 7s - loss: 0.3134 - acc: 0.8547 - val_loss: 0.2514 - val_acc: 0.8969
Epoch 59/120
Epoch 00059: val_loss improved from 0.24870 to 0.24867, saving model to best_m.h5
 - 8s - loss: 0.2661 - acc: 0.8820 - val_loss: 0.2487 - val_acc: 0.8781
Epoch 60/120
Epoch 00060: val_loss improved from 0.24867 to 0.24559, saving model to best_m.h5
 - 8s - loss: 0.2698 - acc: 0.8851 - val_loss: 0.2456 - val_acc: 0.9031
Epoch 61/120
Epoch 00061: val_loss did not improve
 - 7s - loss: 0.2848 - acc: 0.8719 - val_loss: 0.2503 - val_acc: 0.8875
Epoch 62/120
Epoch 00062: val_loss did not improve
 - 7s - loss: 0.2696 - acc: 0.8734 - val_loss: 0.2639 - val_acc: 0.8875
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 7s - loss: 0.2649 - acc: 0.8852 - val_loss: 0.26

In [14]:
with open('../features/resnet_aug4_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/resnet_aug4_sub.csv', index=False)
# deep2 0.227
# deep3 0.223

# pre 2420
# new patience to 30, 2605

0.26059597144
         id  is_iceberg
0  5941774d    0.463484
1  4023181e    0.412983
2  b20200e4    0.129309
3  e7f018bb    0.987797
4  4371c8c3    0.301235
